In [85]:
from Bio.Seq import Seq
from Bio import SeqIO
from fuzzysearch import find_near_matches
import statistics
import logging

# For each read in a fastq file
# find the positions of the primer, could be in forward or reverse orientation
#     TODO: How to deal, look for F first then if none R, or look for both and take highest?
#     TODO: if its the dumbell method, then we will get both reads, so it will be F then R then F etc.
# print the number of positions found and the lengths of the resulting reads, remember the inital one if not at start
fastq_in = "minIon.fastq"
is_dumbell = False
primer_seq = "TCCTCCTCCGTTGTTGTTGTTG"
primer_seq_c = Seq(primer_seq).complement() #also .reverse_complement()
max_d = 2
min_len = 0
max_len = 9999999999
# TODO: add in filtering of the cut reads based on min max, and add summary to output stats.

get_strt = lambda x: str(x).split(', ')[0].split('=')[1] # same as str(matches[1]).split(', ')[0].split('=')[1]
read_count = 0
primer_starts = []
for record in SeqIO.parse(fastq_in, "fastq"):
    read_count += 1
    matches = find_near_matches(primer_seq, record.seq, max_l_dist=max_d)
    matches_comp = find_near_matches(primer_seq_c, record.seq, max_l_dist=max_d)
    if not is_dumbell:
        if len(matches) > 0 and len(matches_comp) > 0:
            if len(matches) >= len(matches_comp):
                logging.info("Found forward and comp primers for read %d, using highest which is forward" % (read_count))
            else:
                logging.info("Found forward and comp primers for read %d, using highest which is complement" % (read_count))
                matches = matches_comp
        elif len(matches) < 2:
            if len(matches_comp) >= 2: # means its comp strand sequenced
                matches = matches_comp
            else:
                if len(matches) == 1 or len(matches_comp) == 1: # means there is only one read
                    primer_starts.append(1)
                    SeqIO.write(record, "read_" + str(read_count) + "_single.fq", "fastq") # this does not append
                else:
                    primer_starts.append(0)
                pass
    else:
        matches = matches + matches_comp
        if len(matches) < 2:
            if len(matches) == 1 : # means there is only one read
                primer_starts.append(1)
                SeqIO.write(record, "read_" + str(read_count) + "_single.fq", "fastq") # this does not append
            else:
                primer_starts.append(0)
            pass
    # So now matches has all the info
    strts = [0] + list(map(int, list(map(get_strt, matches)))) + [len(record.seq)]
    strts.sort() # for the joining in dumbell method
    
    # lets get some stats
    #     rem duplicates if there are because of a perfect primer start and the addition of 0 above
    strts = list(dict.fromkeys(strts))
    read_lngs = [j-i for i, j in zip(strts[:-1], strts[1:])]
    #maybe do mean, sd and median IRQ.
    logging.info("Mean lenth for read %d is %d, with a min of %d, a max of %d before filtering (if user selected)" % (read_count, statistics.mean(read_lngs), min(read_lngs), max(read_lngs)))

    # filter
    if sum(1 for x in read_lngs if min_len <= x <= max_len) < 2:
        primer_starts.append(1)
        ext = "_single.fq"
    else:
        ext = "_multiple.fq"
    
    records_tmp = []
    for i in range(0, len(strts)-1):
        if min_len <= (strts[i+1]-strts[i]) <= max_len:
            records_tmp.append(record[strts[i]:strts[i+1]])

    SeqIO.write(records_tmp, "read_" + str(read_count) + ext, "fastq") # this does not append
    # this can now be passed to the other script to make a consensus
    primer_starts.append(len(strts)-1) #or -2?

# logging.info("")
with_seq = len([i for i in primer_starts if i > 1]) 
print("From the %d input reads, %d had more than one primer start and there was a media of %d primer starts. The mean length of the reads output is XX (still TODO)" % (read_count, with_seq, statistics.median(primer_starts)))

From the 7 input reads, 6 had more than one primer start and there was a media of 2 primer starts. The mean length of the reads output is XX (still TODO)


1
